# Tasks Test 

S_DQ_RET: This is the daily return of individual stocks.

905S_DQ_RET: This is the daily return of the CSI 500 index.

S_DQ_MV: Daily market value (market capitalization) of the stocks.

S_RESTRICT: Indicator if a stock is restricted from being traded the next day. If S_RESTRICT equals 1, the stock cannot be traded the next day.

S_DQ_ADJFACTOR: Adjustment factor for stock prices to account for corporate actions like splits.

S_DQ_VOLUME: Daily trading volume of the stock.

S_DQ_OPEN: Daily opening price of the stock.

S_DQ_CLOSE: Daily closing price of the stock.

FLOAT_A_SHARE: The number of floating shares.

### Fetching Data 

In [12]:
import numpy as np 
import pandas as pd 

In [13]:
data_root = '/Users/huayuzhu/Desktop/exam/raw_data/daily'

In [14]:
def get_data_from_csv(file_path):
    data = pd.read_csv(file_path, index_col=0).T
    data.index = pd.to_datetime(data.index)
    return data

In [80]:
S_DQ_RET = get_data_from_csv(f'{data_root}/S_DQ_RET.csv')
S_905_DQ_RET =  get_data_from_csv(f'{data_root}/905S_DQ_RET.csv')
S_DQ_MV = get_data_from_csv(f'{data_root}/S_DQ_MV.csv')
S_RESTRICT = get_data_from_csv(f'{data_root}/S_RESTRICT.csv')
F7_26 =  get_data_from_csv(f'{data_root}/F7_26.csv')
F7_27 =  get_data_from_csv(f'{data_root}/F7_27.csv')
S_DQ_OPEN = get_data_from_csv(f'{data_root}/S_DQ_OPEN.csv')

In [160]:
S_905_DQ_RET

,905
2015-01-05,0.009581
2015-01-06,0.015121
2015-01-07,0.002009
2015-01-08,-0.010560
2015-01-09,-0.006990
...,...
2022-12-26,0.015373
2022-12-27,0.001998
2022-12-28,-0.004347
2022-12-29,0.005362


In [316]:
class TestInfo: 
    def __init__(self, group_number = 10, trading_frequency = 'W', initial_capital = None, trading_cost =0.0):
        """
        Parameters:
        - group_number(int): Number of groups to divide into (default 10 for deciles).
        - trading_frequency(str): "M" (monthly), "W" (weekly) (default to weekly)
        - initial_capital(float): initial capital plan to invest, default to the capital needed at first period trading 
        - trading cost(float): trading cost percentage, default to 0 

        """
        self.group_number = group_number 
        self.trading_frequency = trading_frequency
        self.initial_capital = initial_capital 
        self.trading_cost = trading_cost 

class OneFactorTest: 
    def __init__(self,test_factor):
        self.factor = test_factor
        self.stock_price_open = S_DQ_OPEN
        self.restricted_stock_df = S_RESTRICT
        self.benchmark = S_905_DQ_RET
    
    def rank_stock(self,test_info): 
        factor_value = self.factor.copy()
        percentile = factor_value.rank(axis=1, pct=True)
        group_number = test_info.group_number - ((1 - percentile) * test_info.group_number) // 1
        return group_number
    
    def trade_interval(self,df,test_info):
        df_factor = df.resample(test_info.trading_frequency).first()   
        df_factor = df_factor.ffill() 
        return df_factor

    def trade(self,test_info): 
        df = self.rank_stock(test_info)
        df_price = self.stock_price_open

        # decide initial capital to trade 
        if test_info.initial_capital is not None: 
            initial_k = test_info.initial_capital
        else:
            p_1_top = df.iloc[0] == 1.0
            p_1_bottom = df.iloc[0] == 10.0
            p_1_top_capital = df_price.iloc[0][p_1_top].sum()
            p_1_bottom_capital = df_price.iloc[0][p_1_bottom].sum()
            initial_k = 1000 * (p_1_bottom_capital + p_1_top_capital)
        side_k = initial_k / 2
        top_pnl_tracker = {'total_pnl': [0], 'cumulative_pnl': [0]}
        top_return_tracker = {'return': [0], 'net_return': [0]}
        bottom_pnl_tracker = {'total_pnl': [0], 'cumulative_pnl': [0]}
        bottom_return_tracker = {'net_return': [0]}
        
        factor_resample = self.trade_interval(df,test_info)
        price_resample = self.trade_interval(df_price, test_info)
        top_r = factor_resample.copy() 
        bottom_r = factor_resample.copy() 
        
        if (factor_resample.index != price_resample.index).any():
            print('please debug')
        for i, (date, signals) in enumerate(factor_resample.iterrows()):
            # here equal weighted portfolio strategy is used, Markovitz, or CAPM optimization may give better result  
            top = signals == 1.0
            bottom = signals == 10.0
            top_k = price_resample.loc[date][top].sum()
            bottom_k = price_resample.loc[date][bottom].sum()
            top_ratio = side_k / top_k if top_k else 0
            bottom_ratio = side_k / bottom_k if bottom_k else 0
            top_r.loc[date] = signals.apply(lambda x: top_ratio if x == 1 else 0)
            bottom_r.loc[date] = signals.apply(lambda x: bottom_ratio if x == 1 else 0)

            # profit calculation 
            if i > 0: 
                price_change = price_resample.iloc[i] - price_resample.iloc[i-1]
                
                top_pnl = (top_r.iloc[i-1] * price_change).sum()
                bottom_pnl = (bottom_r.iloc[i-1] * price_change).sum()
        
                top_cumulative_pnl = top_pnl_tracker['cumulative_pnl'][i-1] + top_pnl
                bottom_cumulative_pnl = bottom_pnl_tracker['cumulative_pnl'][i-1] + bottom_pnl

                top_pnl_tracker['total_pnl'].append(top_pnl)
                top_pnl_tracker['cumulative_pnl'].append(top_cumulative_pnl)

                bottom_pnl_tracker['total_pnl'].append(bottom_pnl)
                bottom_pnl_tracker['cumulative_pnl'].append(bottom_cumulative_pnl)

                top_return_tracker['net_return'].append(top_pnl/side_k - test_info.trading_cost)
                bottom_return_tracker['net_return'].append(bottom_pnl/side_k - test_info.trading_cost)
        df_top_pnl_tracker = pd.DataFrame(top_pnl_tracker, index=factor_resample.index)
        df_top_return_tracker = pd.DataFrame(top_return_tracker, index=factor_resample.index)
        df_top_performance = pd.concat([df_top_pnl_tracker, df_top_return_tracker], axis=1)
        df_top_performance['cumulative_return'] = ((df_top_performance['net_return'] + 1).cumprod() -1)/100

        df_bottom_pnl_tracker = pd.DataFrame(bottom_pnl_tracker, index=factor_resample.index)
        df_bottom_return_tracker = pd.DataFrame(bottom_return_tracker, index=factor_resample.index)
        df_bottom_performance = pd.concat([df_bottom_pnl_tracker, df_bottom_return_tracker], axis=1)
        df_bottom_performance['cumulative_return'] = ((df_bottom_performance['net_return'] + 1).cumprod() -1)/100

        return df_top_performance, df_bottom_performance
    
    def evaluation_mectrics(self,name,performance,adj1 = 48,var=0.05): 
        summary = dict()
        data = performance.loc[:,['net_return']].dropna()
        summary["Annualized Return"] = data.mean() * adj1
        summary["Annualized Volatility"] = data.std() * np.sqrt(adj1)
        summary["Annualized Sharpe Ratio"] = (
            summary["Annualized Return"] / summary["Annualized Volatility"]
        )
        summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
            data[data < 0].std() * np.sqrt(adj1)
        )

        summary["Skewness"] = data.skew()
        summary["Excess Kurtosis"] = data.kurtosis()
        summary[f"VaR ({var})"] = data.quantile(var, axis=0)
        summary[f"CVaR ({var})"] = data[data <= data.quantile(var, axis=0)].mean()
        summary["Min"] = data.min()
        summary["Max"] = data.max()

        wealth_index = 1000 * (1 + data).cumprod()
        previous_peaks = wealth_index.cummax()
        drawdowns = (wealth_index - previous_peaks) / previous_peaks

        summary["Max Drawdown"] = drawdowns.min()

        summary["Bottom"] = drawdowns.idxmin()
        summary["Peak"] = previous_peaks.idxmax()
        return pd.DataFrame(summary) 
    
    def eval_combined(self,test_info): 
        per_top, per_bottom = self.trade(test_info)
        top = self.evaluation_mectrics('top',per_top,adj1 = 48,var=0.05)
        bottom = self.evaluation_mectrics('bottom',per_bottom,adj1 = 48,var=0.05)
        return pd.concat([top,bottom], axis=0,keys=['top', 'bottom'])
    
    def compare_with_benchmark(self,test_info):
        per_top, per_bottom = self.trade(test_info)
        ret_top = per_top.loc[:,['net_return']].dropna()
        ret_bottom = per_bottom.loc[:,['net_return']].dropna()
 

        daily_resampled_top = ret_top.resample('D').ffill().div(5)
        daily_resampled_bottom = ret_bottom.resample('D').ffill().div(5)
        daily_resampled_top = daily_resampled_top.reindex(self.benchmark.index, method='ffill')  
        daily_resampled_bottom = daily_resampled_bottom.reindex(self.benchmark.index, method='ffill') 

        daily_resampled_bottom.fillna(0, inplace= True)
        daily_resampled_top.fillna(0, inplace = True)

        if (self.benchmark.index != daily_resampled_bottom.index).any(): 
            print('please debug')
        summary = pd.DataFrame(index = self.benchmark.index) 
        summary["Daily excess_return TOP"] = (daily_resampled_top['net_return'] - self.benchmark[905])
        summary["Cumulative excess_return TOP"] = (1 + summary['Daily excess_return TOP']).cumprod() - 1
        summary["Daily excess_return BOTTOM"] = (daily_resampled_top['net_return'] - self.benchmark[905])
        summary["Cumulative excess_return BOTTOM"] = (1 + summary['Daily excess_return BOTTOM']).cumprod() - 1

        return summary





In [317]:
test1 = TestInfo()
factor27 = OneFactorTest(F7_27)
factor27.compare_with_benchmark(test1)

,Daily excess_return TOP,Cumulative excess_return TOP,Daily excess_return BOTTOM,Cumulative excess_return BOTTOM
2015-01-05,-0.009581,-0.009581,-0.009581,-0.009581
2015-01-06,-0.015121,-0.024557,-0.015121,-0.024557
2015-01-07,-0.002009,-0.026517,-0.002009,-0.026517
2015-01-08,0.010560,-0.016237,0.010560,-0.016237
2015-01-09,0.006990,-0.009360,0.006990,-0.009360
...,...,...,...,...
2022-12-26,-0.016995,-0.414371,-0.016995,-0.414371
2022-12-27,-0.003620,-0.416490,-0.003620,-0.416490
2022-12-28,0.002724,-0.414901,0.002724,-0.414901
2022-12-29,-0.006984,-0.418987,-0.006984,-0.418987


In [315]:
S_905_DQ_RET.columns

Index([905], dtype='int64')

In [205]:
df2

,total_pnl,cumulative_pnl,net_return,cumulative_return
2015-01-11,0.000000,0.000000,0.000000,0.000000
2015-01-18,33963.600697,33963.600697,0.016542,0.000165
2015-01-25,9195.896303,43159.496999,0.004479,0.000211
2015-02-01,79340.168250,122499.665250,0.038642,0.000606
2015-02-08,-12075.967636,110423.697614,-0.005882,0.000543
...,...,...,...,...
2022-12-04,-58717.405909,-728454.007496,-0.028598,-0.004471
2022-12-11,70769.452901,-657684.554595,0.034468,-0.004280
2022-12-18,-13838.716331,-671523.270926,-0.006740,-0.004318
2022-12-25,-10871.785141,-682395.056067,-0.005295,-0.004349


In [63]:
dft = F7_27.copy()

In [28]:
class one_factor_test: 
    def __init__(self,test_factor):
        if test_factor == 'F7_26':
            self.factor = F7_26
        elif test_factor == 'F7_27':
            self.factor == F7_27
        self.stock_price_open = S_DQ_OPEN
        self.restricted_stock_df = S_RESTRICT
        self.benchmark = S_905_DQ_RET

    def signal(self,x, lower_threshold, upper_threshold):
        if x <= lower_threshold:
            return -1
        elif x >= upper_threshold:
            return 1
        else:
            return 0
        
    def adjust_positions_1(self,df_signal_changed, lookback_period):
        df_signal_changed['date'] = pd.to_datetime(df_signal_changed.index)
        
        if lookback_period in ['M', 'W']:
            # Determine 'period_start' and 'period_end'
            if lookback_period == 'M':
                period_group = [df_signal_changed['date'].dt.year, df_signal_changed['date'].dt.month]
            else:  # lookback_period == 'W'
                period_group = [df_signal_changed['date'].dt.isocalendar().year, df_signal_changed['date'].dt.isocalendar().week]
            
            df_signal_changed['period_start'] = df_signal_changed['date'].groupby(period_group).transform('min')

            df_signal_changed['new_position'] = (df_signal_changed['date'] == df_signal_changed['period_start'])
            df_new_position = df_signal_changed[['new_position']]
            df_signal_changed.drop(columns=['date', 'period_start', 'new_position'], inplace=True)
        
            for c in df_signal_changed:

                df_signal_changed[c] = np.where(df_new_position['new_position']==False, np.nan, df_signal_changed[c])

                df_signal_changed = df_signal_changed.fillna(method='ffill')

                df_signal_changed = df_signal_changed.dropna()

                df_signal_changed['date'] = pd.to_datetime(df_signal_changed.index)
                df_signal_changed['period_end'] = df_signal_changed['date'].groupby(period_group).transform('max')
                df_signal_changed['end_position'] = (df_signal_changed['date'] == df_signal_changed['period_end'])
                df_end_position = df_signal_changed[['end_position']]
                df_signal_changed.drop(columns=['date', 'period_end', 'end_position'], inplace=True)

            for c in df_signal_changed:

                df_signal_changed[c] = np.where(df_end_position['end_position']==True, 0, df_signal_changed[c])

        return df_signal_changed

    def trade(self,test_info): 
        df = self.factor.copy()
        df_price = self.stock_price_open
        # ranking all the tickers, potential look ahead bais, can rank each row every week 
        df = df.rank(axis=1, ascending=False, method='min').fillna(method='ffill').fillna(method='bfill').astype(int)

        num = len(self.trade_factor.columns)
        lower_threshold = int(test_info.percentile * num)
        upper_threshold = num - lower_threshold + 1
        df_signal = df.applymap(lambda x: self.signal(x, lower_threshold, upper_threshold))

        if test_info.initial_capital != 0: 
            initial_k = test_info.initial_capital
        else:
            p_1_top = df_signal.iloc[0] == 1
            p_1_bottom = df_signal.iloc[0] == -1
            p_1_top_capital = df_price.iloc[0][p_1_top].sum()
            p_1_bottom_capital = df_price.iloc[0][p_1_bottom].sum()
            initial_k = 10 * (p_1_bottom_capital + p_1_top_capital)
            side_k = initial_k / 2
        
        capital = initial_k
        cash_tracker = {'cash': [capital]}
        pnl_tracker = {'total_pnl': [0], 'cumulative_pnl': [0]}
        return_tracker = {'return': [0], 'net_return': [0]}

        df_signal_changed = df_signal.copy()
        for date, signals in df_signal.iterrows():
            period_top = signals == 1
            period_bottom = signals == -1
            period_top_capital = df_price.loc[date][period_top].sum()
            period_bottom_capital = df_price.loc[date][period_bottom].sum()
            long_ratio = side_k / period_top_capital if period_top_capital else 0
            short_ratio = side_k / period_bottom_capital if period_bottom_capital else 0
            if len(df_signal_changed.loc[date]) == len(signals.apply(lambda x: self.djust_position_row(x, long_ratio, short_ratio))):
                df_signal_changed.loc[date] = signals.apply(lambda x: self.adjust_position_row(x, long_ratio, short_ratio))
            else: 
                print('please debug')
        for i, (date, signals) in enumerate(df_signal_changed.iterrows()):
            if i == 0: 
                continue 
        
            price_change = df_price.iloc[i] - df_price.iloc[i-1]

            unrealized_pnl = (df_signal_changed.iloc[i-1] * price_change).sum()
        
            #realized_pnl = -abs(side_k * short_ratio) * repo_rate / 252  # Cost for shorting securities
            
            total_pnl =  unrealized_pnl
            cumulative_pnl = pnl_tracker['cumulative_pnl'][i-1] + total_pnl  # Cumulative PnL up to this point

            period_return = total_pnl / initial_k
            #cumulative_return = cumulative_pnl/initial_k   
            net_return = period_return - repo_rate/252
            #pnl_tracker['realized_pnl'].append(realized_pnl)
            #pnl_tracker['unrealized_pnl'].append(unrealized_pnl)
            pnl_tracker['total_pnl'].append(total_pnl)
            pnl_tracker['cumulative_pnl'].append(cumulative_pnl)

            return_tracker['return'].append(period_return)
            return_tracker['net_return'].append(net_return)

        pass 

    def trade_result(self,test_info):
        pass

    def trade_plot(self,test_info): 
        pass

    def summary_statistics(self,test_info): 
        pass 

    def result_analysis(self,test_info): 
        pass 
